In [1]:
%load_ext aiida
%aiida

Loaded AiiDA DB environment - profile name: default.

In [2]:
from aiida import orm
import os
os.sys.path.append("/home/jovyan/work/aiida-openbis/")
from aiida_openbis.utils import bisutils
import numpy as np
from aiida.cmdline.commands import cmd_archive

In [3]:
def find_first_workchain(node):
    """Find the first workchain in the provenance that created the structure node."""
    lastcalling = None
    if isinstance(node, orm.StructureData):
        previous_node = node.creator
    else:
        previous_node = node
    while previous_node is not None:
        lastcalling = previous_node
        previous_node = lastcalling.caller
    if lastcalling is not None:
        return lastcalling.label, lastcalling.pk, lastcalling.description
    return None, None, ""

In [4]:
# Connect to openBIS
session = bisutils.log_in(bisurl="openbis", bisuser="admin", bispasswd="changeit")

In [5]:
# New Simulation Experiment
eln_project_code = "/CARBON_NANOMATERIALS/TRIANGULENE_SPIN_CHAINS"

experiment_code = "TRIANGULENE_SPIN_CHAINS_EXP_5"
new_experiment = session.new_experiment(project = eln_project_code, code = experiment_code, type = 'DEFAULT_EXPERIMENT')
new_experiment.props["$name"] = "Simulations"
new_experiment.save()

# Get STM Simulation Workchain from AiiDA
pk = 62
stm_workchain_node = load_node(pk)
structure_stm = stm_workchain_node.inputs.structure
structure_stm_pk = structure_stm.pk
structure_stm_node = load_node(structure_stm_pk)

# Get Geometry Optimisation Workchain from AiiDA
geoopt_workchain_details = find_first_workchain(structure_stm_node)
geoopt_workchain_pk = geoopt_workchain_details[1]
geoopt_workchain = load_node(geoopt_workchain_pk)

# Get First Geometry from AiiDA
first_structure = geoopt_workchain.inputs.structure
first_structure_ase = first_structure.get_ase()
first_structure_ase.positions # Atoms positions
first_structure_ase.symbols # Atoms Symbols
first_structure_ase.pbc # PBC (X, Y, Z)
first_structure.cell # Cell vectors

# Create Atomistic Models collection
inventory_project_code = "/MATERIALS/ATOMISTIC_MODELS/"
collection_code = "ATOMISTIC_MODELS_EXP_1"
atomistic_models_collection = session.new_collection(project = inventory_project_code, code = collection_code, type = 'COLLECTION')
atomistic_models_collection.props["$name"] = "Atomistic Models"
atomistic_models_collection.save()

# Create Atomistic Model (Not Optimised) in openBIS
first_atomistic_model = session.new_sample(collection = atomistic_models_collection, type='ATOMISTIC_MODEL')
first_atomistic_model.props['$name'] = "Atomistic Model 1"
first_atomistic_model.props['description'] = "A nice atomistic model"
first_atomistic_model.props['pbcx'] = int(first_structure_ase.pbc[0])
first_atomistic_model.props['pbcy'] = int(first_structure_ase.pbc[1])
first_atomistic_model.props['pbcz'] = int(first_structure_ase.pbc[2])
first_atomistic_model.props['geo-is-optimised'] = 0
first_atomistic_model.save()

# Create Simulation in openBIS
geoopt_simulation = session.new_sample(experiment = new_experiment, type='SIMULATION')
geoopt_simulation.props['$name'] = "Simulation GeoOpt"
geoopt_simulation.props['description'] = "Simulation Geometry Optimisation"
# new_simulation.props['simulation-type'] = "Something"
# new_simulation.props['wfm-connection'] = "?"
geoopt_simulation.props['wfm-uuid'] = geoopt_workchain.uuid
geoopt_simulation.save()

# Create Geo Optimisation in openBIS and link it to Simulation and Atomistic Model objects
geoopt_multiplicity = geoopt_workchain.inputs.multiplicity.value
geo_optimisation = session.new_sample(experiment = new_experiment, type='GEO_OPTIMISATION', parents = [geoopt_simulation, first_atomistic_model])
geo_optimisation.props['$name'] = "Simulated Geometry Optimisation"
geo_optimisation.props['multiplicity'] = geoopt_multiplicity
geo_optimisation.save()

# Get Optimised Geometry from AiiDA
optimised_structure = stm_workchain_node.inputs.structure
optimised_structure_ase = optimised_structure.get_ase()
optimised_structure_ase.positions # Atoms positions
optimised_structure_ase.symbols # Atoms Symbols
optimised_structure_ase.pbc # PBC (X, Y, Z)
optimised_structure.cell # Cell vectors

optimised_atomistic_model = session.new_sample(collection = atomistic_models_collection, type = 'ATOMISTIC_MODEL', parents = [geo_optimisation])
optimised_atomistic_model.props['$name'] = "Atomistic Model 2"
optimised_atomistic_model.props['description'] = "A nice atomistic model"
optimised_atomistic_model.props['pbcx'] = int(optimised_structure_ase.pbc[0])
optimised_atomistic_model.props['pbcy'] = int(optimised_structure_ase.pbc[1])
optimised_atomistic_model.props['pbcz'] = int(optimised_structure_ase.pbc[2])
optimised_atomistic_model.props['geo-is-optimised'] = 1
optimised_atomistic_model.save()

# Simulated Scanning Tunneling Microscopy
stm_simulation = session.new_sample(experiment = new_experiment, type = 'SIMULATION')
stm_simulation.props['$name'] = "Simulation STM"
stm_simulation.props['description'] = "Simulation"
stm_simulation.props['wfm-uuid'] = stm_workchain_node.uuid
stm_simulation.save()

stm_simulation_openbis = session.new_sample(experiment = new_experiment, type = 'STM_SIMULATION', parents = [stm_simulation, optimised_atomistic_model])
simulated_stm_uuid = stm_workchain_node.uuid
structure = stm_workchain_node.inputs.structure
dft_params = dict(stm_workchain_node.inputs.dft_params)
stm_params = dict(stm_workchain_node.inputs.stm_params)
stm_simulation_openbis.props['$name'] = "Simulated STM"
stm_simulation_openbis.props['description'] = "A nice simulated STM"
stm_simulation_openbis.props['e-min'] = stm_params['--energy_range'][0]
stm_simulation_openbis.props['e-max'] = stm_params['--energy_range'][1]
stm_simulation_openbis.props['de'] = stm_params['--energy_range'][2]
stm_simulation_openbis.save()

#TODO: How to do this using Python commands?
os.system('verdi archive create stm_simulation.aiida --no-call-calc-backward --no-call-work-backward --no-create-backward -N 391')

stm_simulation_dataset = session.new_dataset(
    type = "RAW_DATA", 
    files = ['/home/jovyan/work/aiida-openbis/Python_Scripts/openBIS_schema.json'],
    sample = stm_simulation_openbis
)
stm_simulation_dataset.save()

experiment successfully created.
experiment successfully created.
sample successfully created.
sample successfully created.
sample successfully created.
sample successfully created.
sample successfully created.
sample successfully created.


Critical: failed to write the archive file. Exception: The output file 'stm_simulation.aiida' already exists


https://openbis/
DataSet successfully created.


attribute,value
code,20231221162622394-259
permId,20231221162622394-259
type,RAW_DATA
kind,PHYSICAL
experiment,/CARBON_NANOMATERIALS/TRIANGULENE_SPIN_CHAINS/TRIANGULENE_SPIN_CHAINS_EXP_5
sample,/CARBON_NANOMATERIALS/TRIANGULENE_SPIN_CHAINS/SIMUL_STM146
parents,[]
children,[]
components,[]
containers,[]


In [6]:
# Code that may be useful in future

# pk = 62
# workchain = load_node(pk)
# structure = workchain.inputs.structure

# all_structures = [structure]
# creator_found = True

# while creator_found:
#     creator = structure.creator
#     if creator is not None:
#         structure = creator.inputs.structure
#         all_structures.append(structure)
#     else:
#         creator_found = False